In [ ]:
"""
=====================================
HIT140 - GROUP 83 - ASSESSMENT 3
Bat vs Rat: Investigation A & B (dataset1 + dataset2 + dataset3 (External Dataset: Moon))
=====================================

Pipeline structure:
1. Dataset1 & Dataset2: already cleaned in Assessment 2 → load directly
2. Dataset3 (Moon): newly collected → clean & wrangle separately
3. Merge all three datasets
4. Feature engineering
5. Descriptive analysis & visualization
6. Inferential analysis (statistical tests, regression) & visualization
"""

In [2]:
# ===========================
# Import required libraries
# ===========================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
import warnings
import math
import os

# Suppress runtime warnings (like division by zero)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [4]:
# ===========================
# DEFINE HYPOTHESES
# ===========================
# Clearly state the hypotheses for Investigation A, B, and Moon effect
hypotheses = {
    "Investigation A": {
        "H1": "Presence of rats reduces bat risk-taking behaviour (lower risk score).",
        "H2": "Presence of rats increases hesitation time before feeding."},
    "Investigation B": {
        "H3": "Bat risk-taking behaviour differs across seasons.",
        "H4": "Hesitation time differs across seasons."},
    "External Moon Effect": {
        "H5": "Moon illumination affects bat hesitation behaviour and risk-taking."}
}

# Print hypotheses for reference
print("===== HYPOTHESES =====")
for inv, hyps in hypotheses.items():
    print(f"\n{inv}:")
    for h, desc in hyps.items():
        print(f"  {h}: {desc}")

===== HYPOTHESES =====

Investigation A:
  H1: Presence of rats reduces bat risk-taking behaviour (lower risk score).
  H2: Presence of rats increases hesitation time before feeding.

Investigation B:
  H3: Bat risk-taking behaviour differs across seasons.
  H4: Hesitation time differs across seasons.

External Moon Effect:
  H5: Moon illumination affects bat hesitation behaviour and risk-taking.


In [5]:
# ===========================
# 1. LOAD & CLEAN DATASETS
# ===========================
# Dataset1 (Bats) and Dataset2 (Rats) were cleaned in Assessment 2
df1 = pd.read_csv("C:/Users/khuep/Downloads/YEN/UNITS/HIT140/Assessment 3/dataset1_cleaned.csv")  # bats dataset
df2 = pd.read_csv("C:/Users/khuep/Downloads/YEN/UNITS/HIT140/Assessment 3/dataset2_cleaned.csv")  # rats dataset

# Dataset3 (Moon) is newly collected ("The location was chosen in Israel due to the source of the research)
df3 = pd.read_csv("C:/Users/khuep/Downloads/YEN/UNITS/HIT140/Assessment 3/dataset3.csv")           # moon dataset

# Rename "Illumination" column in moon dataset for clarity
if "Illumination" in df3.columns:
    df3.rename(columns={"Illumination": "moon_illumination"}, inplace=True)

# Convert time columns to datetime
for df_, col in zip([df1, df2, df3], ["start_time","time","start_time"]):
    df_['datetime'] = pd.to_datetime(df_[col], errors='coerce')

# Floor datetime to minute to avoid merge mismatches
for df_ in [df1, df2, df3]:
    df_['datetime'] = df_['datetime'].dt.floor('min')

if "start_time" in df1.columns:
    df1["datetime"] = pd.to_datetime(df1["start_time"], errors="coerce").dt.floor("min")
if "time" in df2.columns:
    df2["datetime"] = pd.to_datetime(df2["time"], errors="coerce").dt.floor("min")
if "start_time" in df3.columns:
    df3["datetime"] = pd.to_datetime(df3["start_time"], errors="coerce").dt.floor("min")

# Drop missing or duplicated datetime
df1 = df1.dropna(subset=['datetime']).drop_duplicates('datetime')
df2 = df2.dropna(subset=['datetime']).drop_duplicates('datetime')
df3 = df3.dropna(subset=['datetime']).drop_duplicates('datetime')

# Drop hours_after_sunset column of df1 to keep df2's
df1 = df1.drop(columns=["hours_after_sunset"], errors="ignore")

In [7]:
# ===========================
# 2. MERGE ALL DATASETS
# ===========================
# Merge bats, rats, and moon datasets on datetime
# df = df1.merge(df2, on='datetime', how='outer', suffixes=('_bat','_rat')).merge(df3, on='datetime', how='outer')
# df = df.sort_values('datetime').reset_index(drop=True)
df = df1.merge(df2, on="datetime", how="outer", suffixes=("_bat","_rat"))
df = df.merge(df3, on="datetime", how="outer")

# Checking columns
print(df[["rat_arrival_number","hours_after_sunset","season"]].head())


# Print merge statistics to verify
print("\n===== Merge Statistics =====")
print(f"Rows df1 (bats): {len(df1)}")
print(f"Rows df2 (rats): {len(df2)}")
print(f"Rows df3 (moon): {len(df3)}")
print(f"Rows after merging: {len(df)}")

# Fill missing values
# Numeric columns: fill with median to reduce skew from outliers
# Non-numeric columns: fill with "Unknown" to avoid errors in categorical analyses
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col].fillna(df[col].median())
    else:
        df[col] = df[col].fillna("Unknown")

print(f"Total missing values after fill: {df.isna().sum().sum()} (should be 0)")

   rat_arrival_number  hours_after_sunset  season
0                 NaN                 NaN     NaN
1                 0.0                -0.5     NaN
2                 0.0                 0.0     NaN
3                 0.0                 0.5     NaN
4                 0.0                 1.0     NaN

===== Merge Statistics =====
Rows df1 (bats): 628
Rows df2 (rats): 2123
Rows df3 (moon): 158
Rows after merging: 2888
Total missing values after fill: 0 (should be 0)


In [8]:
# ===========================
# 3. FEATURE ENGINEERING
# ===========================
def feature_engineering(df):
    # ---------------------------
    # Convert month to season
    # ---------------------------
    """We convert seasons to Northern Hemisphere months because the two species in this dataset, the Egyptian Fruit Bat (Rousettus aegyptiacus) 
    and the Black Rat (Rattus rattus), live in the Middle East, which is in the Northern Hemisphere."""
    if "month" in df.columns:
        def month_to_season(m):
            return ("Winter" if m in [12,1,2] else
                    "Spring" if m in [3,4,5] else
                    "Summer" if m in [6,7,8] else
                    "Autumn")
        df["season"] = df["month"].apply(month_to_season).astype("category")
    
    # ---------------------------
    # Define rat presence as 0/1
    # ---------------------------
    if {"rat_period_start","rat_period_end","start_time"}.issubset(df.columns):
        df["rat_present"] = ((df["start_time"] >= df["rat_period_start"]) & 
                             (df["start_time"] <= df["rat_period_end"])).astype(int)
    elif "rat_arrival_number" in df.columns:
        df["rat_present"] = (df["rat_arrival_number"] > 0).astype(int)
    
    # ---------------------------
    # Calculate vigilance index (risk * reward)
    # ---------------------------
    if {"risk","reward"}.issubset(df.columns):
        df["vigilance_index"] = df["risk"] * df["reward"]
    
    # ---------------------------
    # Create rat timing bins (relative to arrival)
    # ---------------------------
    if "seconds_after_rat_arrival" in df.columns:
        bins = [-9999,0,30,60,99999]
        labels = ["before_rat","0-30s","30-60s",">60s"]
        df["rat_timing_bin"] = pd.cut(df["seconds_after_rat_arrival"].fillna(99999), bins=bins, labels=labels)
    
    # ---------------------------
    # Rat pressure (rat minutes / bat landings)
    # ---------------------------
    if {"rat_minutes","bat_landing_number"}.issubset(df.columns):
        df["rat_pressure"] = df["rat_minutes"] / (df["bat_landing_number"].replace(0, np.nan))
        df["rat_pressure"] = df["rat_pressure"].fillna(0)
        df["rat_pressure_cat"] = pd.cut(df["rat_pressure"], bins=3, labels=["low","med","high"], duplicates="drop")
    
    # ---------------------------
    # Hesitation level (categorical) safely
    # ---------------------------
    if "bat_landing_to_food" in df.columns:
        vals = df["bat_landing_to_food"]
    try:
        if vals.nunique() >= 2:
            # Let qcut handle duplicate bin edges
            df["hesitation_level"] = pd.qcut(vals, q=min(3, vals.nunique()), duplicates="drop")
            # Convert numeric bins to labels if needed
            bin_edges = df["hesitation_level"].cat.categories
            n_actual_bins = len(bin_edges)
            labels = ["low","medium","high"][:n_actual_bins]
            df["hesitation_level"] = pd.qcut(vals, q=min(3, vals.nunique()), labels=labels, duplicates="drop")
        else:
            df["hesitation_level"] = "medium"
    except ValueError:
        print("Warning: hesitation_level could not be binned. Assigning 'medium' by default.")
        df["hesitation_level"] = "medium"

    df["hesitation_level"] = df["hesitation_level"].reindex(df.index)
    
    # ---------------------------
    # Moon light bins for categorical analysis
    # ---------------------------
    if "moon_illumination" in df.columns:
        df["moon_light"] = pd.cut(df["moon_illumination"], bins=[0,0.33,0.66,1], labels=["dark","half","bright"])
    
    return df

df = feature_engineering(df)

In [9]:
# ===========================
# 4. REMOVE OUTLIERS & FILL MISSING
# ===========================
# Remove outliers in hesitation
if "bat_landing_to_food" in df.columns:
    Q1 = df["bat_landing_to_food"].quantile(0.25)
    Q3 = df["bat_landing_to_food"].quantile(0.75)
    IQR = Q3 - Q1
    df = df[~((df["bat_landing_to_food"] < (Q1 - 1.5*IQR)) | 
              (df["bat_landing_to_food"] > (Q3 + 1.5*IQR)))]

# Fill missing numeric with median
for col in df.select_dtypes(include=np.number).columns:
    df[col] = df[col].fillna(df[col].median())

# Fill missing categorical with "Unknown"
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].fillna("Unknown")

print(f"Total missing values after fill: {df.isna().sum().sum()} (should be 0)")

Total missing values after fill: 0 (should be 0)
